# Project Notes (Sanitized for Git)

This repository contains a **sanitized** version of the Gracity Insects YOLOv8 Classification notebooks.
All tenant-specific identifiers (bucket names, namespaces, OCIDs, local absolute paths) have been replaced by placeholders.

**Author:** Cristina Varas Menadas  
**Last updated:** 2026-02-19

> To run these notebooks, set the configuration values in the first "Configuration" section of each notebook.


# Gracity Insects — 06. Gradio Demo (Upload Image → Predict)

**Goal:** Provide a simple UI to upload an image and run inference with the trained YOLOv8 classification model.

- Works in an OCI Data Science Notebook Session.
- Supports loading weights from a local `best.pt` (recommended) or `best.onnx`.
- If your weights are stored in Object Storage, this notebook can download them using **Resource Principals**.

> Note: In your dataset, `test/` is used as validation.


## 0) Environment & installs

If you already trained in this same kernel/environment, you can skip the installs. Otherwise, run the cell below.


In [ ]:
# If you hit dependency conflicts, prefer using a dedicated conda env for training/inference.
# This cell aims for a practical, "works in notebooks" setup.

!pip -q install --upgrade "ultralytics==8.3.0" "gradio==5.49.1" "opencv-python-headless==4.10.0.84" "pillow>=10.0.0" "numpy<2.0"


## 1) Imports & quick checks


In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Tuple

from PIL import Image

import gradio as gr
from ultralytics import YOLO

print("Ultralytics:", __import__("ultralytics").__version__)


## 2) Configure paths

Update the paths below:

- `RUN_DIR`: your local run folder that contains `weights/best.pt`
- `USE_OBJECT_STORAGE`: set to `True` if you want to download the weights from your bucket


In [ ]:
@dataclass(frozen=True)
class Config:
    # Local training run folder (update this!)
    RUN_DIR: Path = Path("<LOCAL_PATH> Gracity/gracity-insects-yolo-cls/runs/insects_kaggle_cls_1771423582")

    # If True, download weights from Object Storage into LOCAL_WEIGHTS_DIR
    USE_OBJECT_STORAGE: bool = False

    # Object Storage location (only used if USE_OBJECT_STORAGE=True)
    BUCKET_NAME: str = "<BUCKET_NAME>"
    # Prefix to the run folder in the bucket, e.g. ".../yolo/runs/insects_kaggle_v1/<RUN_NAME>/"
    RUNS_PREFIX: str = "<PROJECT_PREFIX>/yolo/runs/insects_kaggle_v1"
    RUN_NAME: str = "insects_kaggle_cls_1771423582"

    LOCAL_WEIGHTS_DIR: Path = Path("./weights_downloaded")

cfg = Config()

# Local paths (preferred)
BEST_PT_LOCAL: Path = cfg.RUN_DIR / "weights" / "best.pt"
BEST_ONNX_LOCAL: Path = cfg.RUN_DIR / "weights" / "best.onnx"

print("best.pt exists:", BEST_PT_LOCAL.exists(), BEST_PT_LOCAL)
print("best.onnx exists:", BEST_ONNX_LOCAL.exists(), BEST_ONNX_LOCAL)


## 3) (Optional) Download weights from Object Storage via Resource Principals

Use this if you uploaded your run artifacts into Object Storage and want to run inference without relying on the local `runs/` directory.


In [ ]:
# Only run if cfg.USE_OBJECT_STORAGE is True
if cfg.USE_OBJECT_STORAGE:
    import oci
    from oci.object_storage import ObjectStorageClient

    signer = oci.auth.signers.get_resource_principals_signer()
    os_client = ObjectStorageClient(config={}, signer=signer)
    namespace = os_client.get_namespace().data

    cfg.LOCAL_WEIGHTS_DIR.mkdir(parents=True, exist_ok=True)

    # Expected objects in bucket:
    # {RUNS_PREFIX}/{RUN_NAME}/weights/best.pt  (or best.onnx)
    base_prefix = f"{cfg.RUNS_PREFIX}/{cfg.RUN_NAME}/weights/"
    candidates = ["best.pt", "best.onnx"]

    downloaded: Dict[str, Path] = {}
    for fname in candidates:
        obj_name = base_prefix + fname
        out_path = cfg.LOCAL_WEIGHTS_DIR / fname
        try:
            resp = os_client.get_object(namespace, cfg.BUCKET_NAME, obj_name)
            with open(out_path, "wb") as f:
                for chunk in resp.data.raw.stream(1024 * 1024, decode_content=False):
                    f.write(chunk)
            downloaded[fname] = out_path
            print("Downloaded:", obj_name, "→", out_path)
        except Exception as e:
            print("Not found or cannot download:", obj_name, "|", repr(e))

    BEST_PT_LOCAL = downloaded.get("best.pt", BEST_PT_LOCAL)
    BEST_ONNX_LOCAL = downloaded.get("best.onnx", BEST_ONNX_LOCAL)

    print("Final best.pt:", BEST_PT_LOCAL, "exists:", BEST_PT_LOCAL.exists())
    print("Final best.onnx:", BEST_ONNX_LOCAL, "exists:", BEST_ONNX_LOCAL.exists())


## 4) Load model

Prefer `best.pt` for simplest inference (PyTorch). If you only have `best.onnx`, it also works.


In [ ]:
def pick_weights(best_pt: Path, best_onnx: Path) -> Path:
    if best_pt.exists():
        return best_pt
    if best_onnx.exists():
        return best_onnx
    raise FileNotFoundError(f"Could not find weights. Looked for: {best_pt} and {best_onnx}")

WEIGHTS_PATH: Path = pick_weights(BEST_PT_LOCAL, BEST_ONNX_LOCAL)
print("Using weights:", WEIGHTS_PATH)

model = YOLO(str(WEIGHTS_PATH))
print("Model loaded. #classes:", len(model.names))


## 5) Prediction function

Returns:
- Top-1 label
- Top-5 labels with probabilities


In [ ]:
def predict(image: Image.Image, topk: int = 5) -> Tuple[str, Dict[str, float]]:
    # Ensure RGB
    if image.mode != "RGB":
        image = image.convert("RGB")

    # Ultralytics accepts PIL directly
    results = model.predict(image, verbose=False)

    probs = results[0].probs  # classification probabilities
    topk = min(topk, len(model.names))

    top_idx = list(map(int, probs.top5[:topk]))
    top_conf = list(map(float, probs.top5conf[:topk]))

    top1_name = model.names[int(probs.top1)]
    scores = {model.names[i]: c for i, c in zip(top_idx, top_conf)}
    return top1_name, scores


## 6) Gradio app (Upload image → Predict)

Run the next cell and open the public link shown in the output.


In [ ]:
def gradio_predict(img: Image.Image) -> Tuple[str, Dict[str, float]]:
    top1, scores = predict(img, topk=5)
    return top1, scores

demo = gr.Interface(
    fn=gradio_predict,
    inputs=gr.Image(type="pil", label="Upload an insect image"),
    outputs=[
        gr.Label(num_top_classes=5, label="Top predictions"),
        gr.JSON(label="Top-5 scores (label → probability)"),
    ],
    title="Gracity Insects — YOLOv8 Classification Demo",
    description="Upload an image and the model returns the predicted class (Top-1) and Top-5 probabilities.",
    allow_flagging="never",
)

demo.launch(share=True)
